In [71]:
from relationGraph import Relation, RelationGraph, MatrixOfRelationGraph
from autoencoder import seedy, AutoEncoder
import utilityFunctions as uf
from main import test_build_relation_graph_with_symertic_data, test_convert_graph_to_2D_matrix, test_get_matix_for_autoencoder, test_autoencoder
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from base import load_source
from os.path import join
import numpy as np


from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Masking, UpSampling2D, Dense, Reshape, Flatten, Concatenate, concatenate
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

In [2]:
input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

autoencoder.summary()

# autoencoder.fit(x_train, x_train,
#                 epochs=50,
#                 batch_size=128,
#                 shuffle=True,
#                 validation_data=(x_test, x_test))

decoded_imgs = autoencoder.predict(x_test)

mse = mean_squared_error(x_test[0].flatten(), decoded_imgs[0].flatten())
print(mse)

11493376/11490434 [==============================] - 0s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 8)         1160      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 8)           0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 8)           584       
_________________________________________________________________
max_pooling

In [27]:
num = 162
fn = '/data/samples/dicty/' + str(num) + '_org_data.npz'
f = np.load(fn)
data = f[f.files[0]]
x, _ = data.shape
data = data.reshape(x, num, num, 1)

()


In [28]:
input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format
x = Conv2D(1, (1, 28), activation='relu')(input_img)

x = UpSampling2D((1, 28))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

autoencoder.summary()

autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test, x_test))

decoded_imgs = autoencoder.predict(x_test)

mse = mean_squared_error(x_test[0].flatten(), decoded_imgs[0].flatten())
print(mse)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_61 (Conv2D)           (None, 28, 1, 1)          29        
_________________________________________________________________
up_sampling2d_11 (UpSampling (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 28, 28, 1)         10        
Total params: 39
Trainable params: 39
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 3s 49us/step - loss: 0.5869 - val_loss: 0.4993
Epoch 2/50
60000/60000 [==============================] - 2s 38us/step - loss: 0.4476 - val_loss: 0.4140
Epoch 3/50
60000/6

In [139]:
# # first input model
# visible1 = Input(shape=(28,28,1))
# conv11 = Conv2D(1, (1,28), activation='relu')(visible1)
# flat1 = Flatten()(conv11)

# # second input model
# visible2 = Input(shape=(28,28,1))
# conv21 = Conv2D(1, (1, 28), activation='relu')(visible2)
# flat2 = Flatten()(conv21)

# # merge input models
# x = concatenate([flat1, flat2])

# # autoencoder
# x = Dense(28, activation='relu')(x)
# x = Dense(20, activation='relu')(x)
# x = Dense(10, activation='relu')(x)
# x = Dense(20, activation='relu')(x)
# x = Dense(28, activation='relu')(x)

# # first output model
# y1 = Reshape((28,1,1))(x)
# y1 = UpSampling2D((1, 28))(y1)
# y1 = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(y1)

# # secound output model
# y2 = Reshape((28,1,1))(x)
# y2 = UpSampling2D((1, 28))(y2)
# y2 = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(y2)

# autoencoder = Model(inputs=[visible1, visible2], outputs=[y1, y2])
# autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

def inputs():
    visible1 = Input(shape=(28,28,1))
    conv11 = Conv2D(1, (1,28), activation='relu')(visible1)
    flat1 = Flatten()(conv11)
    
    return flat1, visible1

def outputs(inputs):
    y1 = Dense(28, activation='relu')(inputs)
    y1 = Reshape((28,1,1))(y1)
    y1 = UpSampling2D((1, 28))(y1)
    y1 = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(y1)
    
    return y1

def autoencoder(inputs):
    x = Dense(28, activation='relu')(inputs)
    x = Dense(20, activation='relu')(x)
    x = Dense(10, activation='relu')(x)
    x = Dense(20, activation='relu')(x)
#     x = Dense(28, activation='relu')(x)
    
    return x


input_layers = []
visible_inputs = []

# first input model
layers, visible = inputs()
input_layers.append(layers)
visible_inputs.append(visible)

# secound input model
layers, visible = inputs()
input_layers.append(layers)
visible_inputs.append(visible)


# merge input models
x = concatenate(input_layers)

outputs_layers = []
# first output model
outputs_layers.append(outputs(x))
# secound output model
outputs_layers.append(outputs(x))

autoencoder = Model(inputs=visible_inputs, outputs=outputs_layers)
autoencoder.compile(optimizer='sgd', loss='mse')

(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

autoencoder.summary()

log_dir = '/data/logs/'
callbacks = [
            TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=True),
            EarlyStopping(monitor='loss', min_delta=0, patience=3, verbose=0, mode='auto')
        ]

x_train1 = x_train[:int(len(x_train)/2)]
x_train2 = x_train[int(len(x_train)/2):]

x_test1 = x_test[:int(len(x_test)/2)]
x_test2 = x_test[int(len(x_test)/2):]



autoencoder.fit([x_train1, x_train2], [x_train1, x_train2],
                epochs=300,
                batch_size=200,
                shuffle=True,
                callbacks=callbacks,
                validation_data=([x_test1, x_test2], [x_test1, x_test2])
               )

decoded_imgs = autoencoder.predict([x_test1, x_test2])

mse = mean_squared_error(x_test1.flatten(), decoded_imgs[0].flatten())
print(mse)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_194 (InputLayer)          (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
input_195 (InputLayer)          (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
conv2d_342 (Conv2D)             (None, 28, 1, 1)     29          input_194[0][0]                  
__________________________________________________________________________________________________
conv2d_343 (Conv2D)             (None, 28, 1, 1)     29          input_195[0][0]                  
__________________________________________________________________________________________________
flatten_18

30000/30000 [==============================] - 1s 41us/step - loss: 0.1932 - conv2d_344_loss: 0.0974 - conv2d_345_loss: 0.0958 - val_loss: 0.1956 - val_conv2d_344_loss: 0.0911 - val_conv2d_345_loss: 0.1045
Epoch 23/300
30000/30000 [==============================] - 1s 36us/step - loss: 0.1928 - conv2d_344_loss: 0.0972 - conv2d_345_loss: 0.0956 - val_loss: 0.1953 - val_conv2d_344_loss: 0.0909 - val_conv2d_345_loss: 0.1044
Epoch 24/300
30000/30000 [==============================] - 1s 40us/step - loss: 0.1924 - conv2d_344_loss: 0.0970 - conv2d_345_loss: 0.0954 - val_loss: 0.1950 - val_conv2d_344_loss: 0.0907 - val_conv2d_345_loss: 0.1043
Epoch 25/300
30000/30000 [==============================] - 1s 38us/step - loss: 0.1922 - conv2d_344_loss: 0.0969 - conv2d_345_loss: 0.0953 - val_loss: 0.1947 - val_conv2d_344_loss: 0.0905 - val_conv2d_345_loss: 0.1042
Epoch 26/300
30000/30000 [==============================] - 1s 42us/step - loss: 0.1919 - conv2d_344_loss: 0.0968 - conv2d_345_loss: 0.09

30000/30000 [==============================] - 1s 41us/step - loss: 0.1898 - conv2d_344_loss: 0.0958 - conv2d_345_loss: 0.0941 - val_loss: 0.1928 - val_conv2d_344_loss: 0.0890 - val_conv2d_345_loss: 0.1038
Epoch 60/300
30000/30000 [==============================] - 1s 39us/step - loss: 0.1898 - conv2d_344_loss: 0.0958 - conv2d_345_loss: 0.0941 - val_loss: 0.1928 - val_conv2d_344_loss: 0.0890 - val_conv2d_345_loss: 0.1038
Epoch 61/300
30000/30000 [==============================] - 1s 35us/step - loss: 0.1898 - conv2d_344_loss: 0.0957 - conv2d_345_loss: 0.0941 - val_loss: 0.1928 - val_conv2d_344_loss: 0.0890 - val_conv2d_345_loss: 0.1038
Epoch 62/300
30000/30000 [==============================] - 1s 42us/step - loss: 0.1898 - conv2d_344_loss: 0.0957 - conv2d_345_loss: 0.0941 - val_loss: 0.1928 - val_conv2d_344_loss: 0.0889 - val_conv2d_345_loss: 0.1038
Epoch 63/300
30000/30000 [==============================] - 1s 41us/step - loss: 0.1898 - conv2d_344_loss: 0.0957 - conv2d_345_loss: 0.09

30000/30000 [==============================] - 1s 35us/step - loss: 0.1896 - conv2d_344_loss: 0.0956 - conv2d_345_loss: 0.0940 - val_loss: 0.1926 - val_conv2d_344_loss: 0.0888 - val_conv2d_345_loss: 0.1038
Epoch 97/300
30000/30000 [==============================] - 1s 36us/step - loss: 0.1896 - conv2d_344_loss: 0.0956 - conv2d_345_loss: 0.0940 - val_loss: 0.1926 - val_conv2d_344_loss: 0.0888 - val_conv2d_345_loss: 0.1038
Epoch 98/300
30000/30000 [==============================] - 1s 31us/step - loss: 0.1896 - conv2d_344_loss: 0.0956 - conv2d_345_loss: 0.0940 - val_loss: 0.1926 - val_conv2d_344_loss: 0.0888 - val_conv2d_345_loss: 0.1038
Epoch 99/300
30000/30000 [==============================] - 1s 33us/step - loss: 0.1896 - conv2d_344_loss: 0.0956 - conv2d_345_loss: 0.0940 - val_loss: 0.1926 - val_conv2d_344_loss: 0.0888 - val_conv2d_345_loss: 0.1038
Epoch 100/300
30000/30000 [==============================] - 1s 30us/step - loss: 0.1896 - conv2d_344_loss: 0.0956 - conv2d_345_loss: 0.0

30000/30000 [==============================] - 1s 41us/step - loss: 0.1894 - conv2d_344_loss: 0.0955 - conv2d_345_loss: 0.0939 - val_loss: 0.1924 - val_conv2d_344_loss: 0.0886 - val_conv2d_345_loss: 0.1038
Epoch 134/300
30000/30000 [==============================] - 1s 37us/step - loss: 0.1894 - conv2d_344_loss: 0.0955 - conv2d_345_loss: 0.0939 - val_loss: 0.1924 - val_conv2d_344_loss: 0.0886 - val_conv2d_345_loss: 0.1038
Epoch 135/300
30000/30000 [==============================] - 1s 36us/step - loss: 0.1894 - conv2d_344_loss: 0.0955 - conv2d_345_loss: 0.0939 - val_loss: 0.1924 - val_conv2d_344_loss: 0.0886 - val_conv2d_345_loss: 0.1038
Epoch 136/300
30000/30000 [==============================] - 1s 41us/step - loss: 0.1894 - conv2d_344_loss: 0.0955 - conv2d_345_loss: 0.0939 - val_loss: 0.1924 - val_conv2d_344_loss: 0.0886 - val_conv2d_345_loss: 0.1038
Epoch 137/300
30000/30000 [==============================] - 1s 35us/step - loss: 0.1893 - conv2d_344_loss: 0.0955 - conv2d_345_loss: 

30000/30000 [==============================] - 1s 42us/step - loss: 0.1889 - conv2d_344_loss: 0.0952 - conv2d_345_loss: 0.0937 - val_loss: 0.1920 - val_conv2d_344_loss: 0.0884 - val_conv2d_345_loss: 0.1036
Epoch 171/300
30000/30000 [==============================] - 1s 42us/step - loss: 0.1889 - conv2d_344_loss: 0.0952 - conv2d_345_loss: 0.0937 - val_loss: 0.1919 - val_conv2d_344_loss: 0.0884 - val_conv2d_345_loss: 0.1036
Epoch 172/300
30000/30000 [==============================] - 1s 39us/step - loss: 0.1889 - conv2d_344_loss: 0.0952 - conv2d_345_loss: 0.0937 - val_loss: 0.1919 - val_conv2d_344_loss: 0.0884 - val_conv2d_345_loss: 0.1036
Epoch 173/300
30000/30000 [==============================] - 1s 42us/step - loss: 0.1889 - conv2d_344_loss: 0.0952 - conv2d_345_loss: 0.0937 - val_loss: 0.1919 - val_conv2d_344_loss: 0.0883 - val_conv2d_345_loss: 0.1036
Epoch 174/300
30000/30000 [==============================] - 1s 35us/step - loss: 0.1888 - conv2d_344_loss: 0.0951 - conv2d_345_loss: 

30000/30000 [==============================] - 1s 36us/step - loss: 0.1880 - conv2d_344_loss: 0.0946 - conv2d_345_loss: 0.0934 - val_loss: 0.1911 - val_conv2d_344_loss: 0.0879 - val_conv2d_345_loss: 0.1032
Epoch 208/300
30000/30000 [==============================] - 1s 40us/step - loss: 0.1880 - conv2d_344_loss: 0.0946 - conv2d_345_loss: 0.0934 - val_loss: 0.1911 - val_conv2d_344_loss: 0.0879 - val_conv2d_345_loss: 0.1032
Epoch 209/300
30000/30000 [==============================] - 1s 41us/step - loss: 0.1879 - conv2d_344_loss: 0.0945 - conv2d_345_loss: 0.0934 - val_loss: 0.1911 - val_conv2d_344_loss: 0.0879 - val_conv2d_345_loss: 0.1032
Epoch 210/300
30000/30000 [==============================] - 1s 38us/step - loss: 0.1879 - conv2d_344_loss: 0.0945 - conv2d_345_loss: 0.0934 - val_loss: 0.1910 - val_conv2d_344_loss: 0.0878 - val_conv2d_345_loss: 0.1032
Epoch 211/300
30000/30000 [==============================] - 1s 33us/step - loss: 0.1879 - conv2d_344_loss: 0.0945 - conv2d_345_loss: 

30000/30000 [==============================] - 1s 40us/step - loss: 0.1864 - conv2d_344_loss: 0.0935 - conv2d_345_loss: 0.0929 - val_loss: 0.1897 - val_conv2d_344_loss: 0.0870 - val_conv2d_345_loss: 0.1026
Epoch 245/300
30000/30000 [==============================] - 1s 36us/step - loss: 0.1863 - conv2d_344_loss: 0.0934 - conv2d_345_loss: 0.0929 - val_loss: 0.1896 - val_conv2d_344_loss: 0.0870 - val_conv2d_345_loss: 0.1026
Epoch 246/300
30000/30000 [==============================] - 1s 41us/step - loss: 0.1862 - conv2d_344_loss: 0.0934 - conv2d_345_loss: 0.0929 - val_loss: 0.1896 - val_conv2d_344_loss: 0.0870 - val_conv2d_345_loss: 0.1026
Epoch 247/300
30000/30000 [==============================] - 1s 42us/step - loss: 0.1862 - conv2d_344_loss: 0.0933 - conv2d_345_loss: 0.0928 - val_loss: 0.1895 - val_conv2d_344_loss: 0.0869 - val_conv2d_345_loss: 0.1026
Epoch 248/300
30000/30000 [==============================] - 1s 42us/step - loss: 0.1861 - conv2d_344_loss: 0.0933 - conv2d_345_loss: 

30000/30000 [==============================] - 1s 41us/step - loss: 0.1837 - conv2d_344_loss: 0.0919 - conv2d_345_loss: 0.0918 - val_loss: 0.1872 - val_conv2d_344_loss: 0.0859 - val_conv2d_345_loss: 0.1013
Epoch 282/300
30000/30000 [==============================] - 1s 40us/step - loss: 0.1836 - conv2d_344_loss: 0.0919 - conv2d_345_loss: 0.0917 - val_loss: 0.1872 - val_conv2d_344_loss: 0.0859 - val_conv2d_345_loss: 0.1013
Epoch 283/300
30000/30000 [==============================] - 1s 41us/step - loss: 0.1836 - conv2d_344_loss: 0.0919 - conv2d_345_loss: 0.0917 - val_loss: 0.1871 - val_conv2d_344_loss: 0.0859 - val_conv2d_345_loss: 0.1012
Epoch 284/300
30000/30000 [==============================] - 1s 38us/step - loss: 0.1835 - conv2d_344_loss: 0.0918 - conv2d_345_loss: 0.0917 - val_loss: 0.1870 - val_conv2d_344_loss: 0.0858 - val_conv2d_345_loss: 0.1012
Epoch 285/300
30000/30000 [==============================] - 1s 35us/step - loss: 0.1834 - conv2d_344_loss: 0.0918 - conv2d_345_loss: 

In [76]:
# Multiple Inputs
# from keras.utils import plot_model
# from keras.models import Model
# from keras.layers import Input
# from keras.layers import Dense
# from keras.layers import Flatten
# from keras.layers.convolutional import Conv2D
# from keras.layers.pooling import MaxPooling2D
# from keras.layers.merge import concatenate
# first input model
visible1 = Input(shape=(64,64,1))
conv11 = Conv2D(32, kernel_size=4, activation='relu')(visible1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(16, kernel_size=4, activation='relu')(pool11)
pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
flat1 = Flatten()(pool12)
# second input model
visible2 = Input(shape=(32,32,3))
conv21 = Conv2D(32, kernel_size=4, activation='relu')(visible2)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
conv22 = Conv2D(16, kernel_size=4, activation='relu')(pool21)
pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
flat2 = Flatten()(pool22)
# merge input models
merge = concatenate([flat1, flat2])
# interpretation model
hidden1 = Dense(10, activation='relu')(merge)
hidden2 = Dense(10, activation='relu')(hidden1)
output = Dense(1, activation='sigmoid')(hidden2)
model = Model(inputs=[visible1, visible2], outputs=output)
# summarize layers
print(model.summary())
# plot graph
# plot_model(model, to_file='multiple_inputs.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_65 (InputLayer)           (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
input_66 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_131 (Conv2D)             (None, 61, 61, 32)   544         input_65[0][0]                   
__________________________________________________________________________________________________
conv2d_133 (Conv2D)             (None, 29, 29, 32)   1568        input_66[0][0]                   
__________________________________________________________________________________________________
max_poolin